# Dataframe to Shot map data

This jupyternotebook presents the functions we implemented to convert the data for the shotmap visualization

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../datasets/tidy_data.csv')

In [3]:
#pd.set_option("max_columns", 5)
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)

df.drop(['Unnamed: 0'], axis=1).head(5)


,event_Idx,period,periodTime,id_team_that_shot,name_team_that_shot,result_event,x_coord,y_coord,rinkSide_of_the_team_that_shot,goalie_name,shooter_name,shot_type,empty_net,strength,type_season,id_game,season,dateTime,endDateTime,abstractGameState,team_away_name,team_home_name
0,5,1,00:16,8,Montréal Canadiens,Shot,-8.0,-36.0,right,Henrik Lundqvist,Dwight King,Wrist Shot,NaN,NaN,playoffs,2016030111,20162017,2017-04-12T23:00:00Z,2017-04-13T01:41:46Z,Final,New York Rangers,Montréal Canadiens
1,11,1,01:56,3,New York Rangers,Shot,85.0,-6.0,left,Carey Price,Rick Nash,Wrist Shot,NaN,NaN,playoffs,2016030111,20162017,2017-04-12T23:00:00Z,2017-04-13T01:41:46Z,Final,New York Rangers,Montréal Canadiens
2,12,1,02:04,8,Montréal Canadiens,Shot,-69.0,-35.0,right,Henrik Lundqvist,Max Pacioretty,Wrist Shot,NaN,NaN,playoffs,2016030111,20162017,2017-04-12T23:00:00Z,2017-04-13T01:41:46Z,Final,New York Rangers,Montréal Canadiens
3,21,1,03:40,8,Montréal Canadiens,Shot,-55.0,-17.0,right,Henrik Lundqvist,Dwight King,Slap Shot,NaN,NaN,playoffs,2016030111,20162017,2017-04-12T23:00:00Z,2017-04-13T01:41:46Z,Final,New York Rangers,Montréal Canadiens
4,32,1,04:56,8,Montréal Canadiens,Shot,-58.0,-28.0,right,Henrik Lundqvist,Paul Byron,Wrist Shot,NaN,NaN,playoffs,2016030111,20162017,2017-04-12T23:00:00Z,2017-04-13T01:41:46Z,Final,New York Rangers,Montréal Canadiens


In [4]:
def binning_x_and_y(df):
    step = 10
    bins_x = [i for i in range(-(100+(step//2)),110+(+(step//2)),step)]
    bin_centers_x = [i+(step//2) for i in bins_x][:-1]

    bins_y = [i for i in range(-(50+(step//2)),60+(step//2),step)]
    bin_centers_y = [i+(step//2) for i in bins_y][:-1]
    df_bis = df.copy()
    df_bis['x_coord_bin'] = pd.cut(df['x_coord'], bins=bins_x, labels=bin_centers_x)
    df_bis['y_coord_bin'] = pd.cut(df['y_coord'], bins=bins_y, labels=bin_centers_y)
    return df_bis

In [5]:
def central_pivot(df_season):
    def pivot(x):
        return -x
    side_to_rotate = 'left'
    if side_to_rotate == 'left':
        df_season.loc[(df_season['rinkSide_of_the_team_that_shot'].isna()) & (df_season['x_coord'] > 0),'rinkSide_of_the_team_that_shot'] = side_to_rotate 
    else:
        df_season.loc[(df_season['rinkSide_of_the_team_that_shot'].isna()) & (df_season['x_coord'] < 0),'rinkSide_of_the_team_that_shot'] = side_to_rotate 
        
    df_season.loc[df_season['rinkSide_of_the_team_that_shot']==side_to_rotate,'x_coord'] = df_season[df_season['rinkSide_of_the_team_that_shot']==side_to_rotate]['x_coord'].apply(pivot)
    df_season.loc[df_season['rinkSide_of_the_team_that_shot']==side_to_rotate,'y_coord'] = df_season[df_season['rinkSide_of_the_team_that_shot']==side_to_rotate]['y_coord'].apply(pivot)
    return df_season

In [6]:
season = 20162017
df_20162017 = df[df["season"]==season]

In [7]:
df_20162017 = central_pivot(df_20162017)
df_20162017 = binning_x_and_y(df_20162017)


In [8]:
df_20162017.head(5)

,Unnamed: 0,event_Idx,period,periodTime,id_team_that_shot,name_team_that_shot,result_event,x_coord,y_coord,rinkSide_of_the_team_that_shot,goalie_name,shooter_name,shot_type,empty_net,strength,type_season,id_game,season,dateTime,endDateTime,abstractGameState,team_away_name,team_home_name,x_coord_bin,y_coord_bin
0,0,5,1,00:16,8,Montréal Canadiens,Shot,-8.0,-36.0,right,Henrik Lundqvist,Dwight King,Wrist Shot,NaN,NaN,playoffs,2016030111,20162017,2017-04-12T23:00:00Z,2017-04-13T01:41:46Z,Final,New York Rangers,Montréal Canadiens,-10,-40
1,1,11,1,01:56,3,New York Rangers,Shot,-85.0,6.0,left,Carey Price,Rick Nash,Wrist Shot,NaN,NaN,playoffs,2016030111,20162017,2017-04-12T23:00:00Z,2017-04-13T01:41:46Z,Final,New York Rangers,Montréal Canadiens,-90,10
2,2,12,1,02:04,8,Montréal Canadiens,Shot,-69.0,-35.0,right,Henrik Lundqvist,Max Pacioretty,Wrist Shot,NaN,NaN,playoffs,2016030111,20162017,2017-04-12T23:00:00Z,2017-04-13T01:41:46Z,Final,New York Rangers,Montréal Canadiens,-70,-40
3,3,21,1,03:40,8,Montréal Canadiens,Shot,-55.0,-17.0,right,Henrik Lundqvist,Dwight King,Slap Shot,NaN,NaN,playoffs,2016030111,20162017,2017-04-12T23:00:00Z,2017-04-13T01:41:46Z,Final,New York Rangers,Montréal Canadiens,-60,-20
4,4,32,1,04:56,8,Montréal Canadiens,Shot,-58.0,-28.0,right,Henrik Lundqvist,Paul Byron,Wrist Shot,NaN,NaN,playoffs,2016030111,20162017,2017-04-12T23:00:00Z,2017-04-13T01:41:46Z,Final,New York Rangers,Montréal Canadiens,-60,-30


In [67]:
#df_20162017[(df_20162017["period"]==2) & (df_20162017["id_game"]==2019030414)]# ["rinkSide_of_the_team_that_shot"]

In [102]:
df_question_2 = df_20162017.groupby(['x_coord','y_coord'])['x_coord'].count().reset_index(name="count")
df_question_2 

,x_coord,y_coord,count
0,-98.0,3.0,1
1,-98.0,9.0,1
2,-98.0,15.0,1
3,-97.0,-18.0,2
4,-97.0,-10.0,2
...,...,...,...
7305,95.0,33.0,1
7306,96.0,-4.0,1
7307,96.0,8.0,1
7308,97.0,-21.0,1


In [9]:
df_question_2 = df_20162017.groupby(['x_coord_bin','y_coord_bin'])['x_coord_bin'].count().reset_index(name="count")
df_question_2 

,x_coord_bin,y_coord_bin,count
0,-100,-50,0
1,-100,-40,0
2,-100,-30,3
3,-100,-20,4
4,-100,-10,6
...,...,...,...
226,100,10,1
227,100,20,0
228,100,30,0
229,100,40,0


In [10]:
df_question_2 = df_20162017.groupby(['x_coord_bin','y_coord_bin'])['x_coord_bin'].count().reset_index(name="count")
df_question_2["mean_shots_per_hour_in_whole_league"] = df_question_2["count"]/df_20162017["id_game"].nunique() 


In [11]:
# count per team per emplacement
df_question_3 = df_20162017.groupby(['x_coord_bin','y_coord_bin','name_team_that_shot'])['name_team_that_shot'].count().reset_index(name="count_per_team")
df_question_3_p = df_question_3.pivot(index=["x_coord_bin","y_coord_bin"], columns="name_team_that_shot", values="count_per_team")
#df_question_3_p[df_question_3_p.isna()]=0
df_question_3_p.head(5)

name_team_that_shot      Anaheim Ducks  Arizona Coyotes  Boston Bruins  \
x_coord_bin y_coord_bin                                                  
-100        -50                      0                0              0   
            -40                      0                0              0   
            -30                      0                0              0   
            -20                      1                0              0   
            -10                      0                0              0   

name_team_that_shot      Buffalo Sabres  Calgary Flames  Carolina Hurricanes  \
x_coord_bin y_coord_bin                                                        
-100        -50                       0               0                    0   
            -40                       0               0                    0   
            -30                       0               0                    0   
            -20                       0               0                    0   
            -10                       0               0                    0   

name_team_that_shot      Chicago Blackhawks  Colorado Avalanche  \
x_coord_bin y_coord_bin                                           
-100        -50                           0                   0   
            -40                           0                   0   
            -30                           0                   0   
            -20                           0                   0   
            -10                           0                   0   

name_team_that_shot      Columbus Blue Jackets  Dallas Stars  \
x_coord_bin y_coord_bin                                        
-100        -50                              0             0   
            -40                              0             0   
            -30                              0             0   
            -20                              0             1   
            -10                              0             0   

name_team_that_shot      Detroit Red Wings  Edmonton Oilers  Florida Panthers  \
x_coord_bin y_coord_bin                                                         
-100        -50                          0                0                 0   
            -40                          0                0                 0   
            -30                          0                0                 0   
            -20                          0                0                 0   
            -10                          0                1                 0   

name_team_that_shot      Los Angeles Kings  Minnesota Wild  \
x_coord_bin y_coord_bin                                      
-100        -50                          0               0   
            -40                          0               0   
            -30                          0               0   
            -20                          0               2   
            -10                          0               0   

name_team_that_shot      Montréal Canadiens  Nashville Predators  \
x_coord_bin y_coord_bin                                            
-100        -50                           0                    0   
            -40                           0                    0   
            -30                           1                    0   
            -20                           0                    0   
            -10                           3                    0   

name_team_that_shot      New Jersey Devils  New York Islanders  \
x_coord_bin y_coord_bin                                          
-100        -50                          0                   0   
            -40                          0                   0   
            -30                          0                   0   
            -20                          0                   0   
            -10                          1                   0   

name_team_that_shot      New York Rangers  Ottawa Senators

In [12]:
df_count_game_per_team = df_20162017.groupby(['name_team_that_shot'])["id_game"].nunique().reset_index(name="count_game_per_team")
df_count_game_per_team.head(5)

,name_team_that_shot,count_game_per_team
0,Anaheim Ducks,99
1,Arizona Coyotes,82
2,Boston Bruins,88
3,Buffalo Sabres,82
4,Calgary Flames,86


In [13]:
for team in df_question_3_p.columns :
    df_question_3_p[team] /= df_count_game_per_team[df_count_game_per_team["name_team_that_shot"]==team]["count_game_per_team"].values

In [14]:
df_question_3_p.head(5) # moyenne des tirs par heure pour chaque équipe 
#df_question_3_p[(df_question_2["x_coord_bin"]==-95) & (df_question_2["y_coord_bin"]==-15)]

name_team_that_shot      Anaheim Ducks  Arizona Coyotes  Boston Bruins  \
x_coord_bin y_coord_bin                                                  
-100        -50               0.000000              0.0            0.0   
            -40               0.000000              0.0            0.0   
            -30               0.000000              0.0            0.0   
            -20               0.010101              0.0            0.0   
            -10               0.000000              0.0            0.0   

name_team_that_shot      Buffalo Sabres  Calgary Flames  Carolina Hurricanes  \
x_coord_bin y_coord_bin                                                        
-100        -50                     0.0             0.0                  0.0   
            -40                     0.0             0.0                  0.0   
            -30                     0.0             0.0                  0.0   
            -20                     0.0             0.0                  0.0   
            -10                     0.0             0.0                  0.0   

name_team_that_shot      Chicago Blackhawks  Colorado Avalanche  \
x_coord_bin y_coord_bin                                           
-100        -50                         0.0                 0.0   
            -40                         0.0                 0.0   
            -30                         0.0                 0.0   
            -20                         0.0                 0.0   
            -10                         0.0                 0.0   

name_team_that_shot      Columbus Blue Jackets  Dallas Stars  \
x_coord_bin y_coord_bin                                        
-100        -50                            0.0      0.000000   
            -40                            0.0      0.000000   
            -30                            0.0      0.000000   
            -20                            0.0      0.012195   
            -10                            0.0      0.000000   

name_team_that_shot      Detroit Red Wings  Edmonton Oilers  Florida Panthers  \
x_coord_bin y_coord_bin                                                         
-100        -50                        0.0         0.000000               0.0   
            -40                        0.0         0.000000               0.0   
            -30                        0.0         0.000000               0.0   
            -20                        0.0         0.000000               0.0   
            -10                        0.0         0.010526               0.0   

name_team_that_shot      Los Angeles Kings  Minnesota Wild  \
x_coord_bin y_coord_bin                                      
-100        -50                        0.0        0.000000   
            -40                        0.0        0.000000   
            -30                        0.0        0.000000   
            -20                        0.0        0.022989   
            -10                        0.0        0.000000   

name_team_that_shot      Montréal Canadiens  Nashville Predators  \
x_coord_bin y_coord_bin                                            
-100        -50                    0.000000                  0.0   
            -40                    0.000000                  0.0   
            -30                    0.011364                  0.0   
            -20                    0.000000                  0.0   
            -10                    0.034091                  0.0   

name_team_that_shot      New Jersey Devils  New York Islanders  \
x_coord_bin y_coord_bin                                          
-100        -50                   0.000000                 0.0   
            -40                   0.000000                 0.0   
            -30                   0.000000                 0.0   
            -20                   0.000000                 0.0   
            -10                   0.012195                 0.0   

name_team_that_shot      New York Rangers  Ottawa Senators

In [15]:
for x,y in  df_question_3_p.index:
    alpha = df_question_2.query(f'x_coord_bin == {x} and y_coord_bin == {y}')["mean_shots_per_hour_in_whole_league"].values
    #if alpha==0.0:
    #    df_question_3_p[(df_question_3_p.index.get_level_values('x_coord_bin')==x) & (df_question_3_p.index.get_level_values('y_coord_bin')==y)] = int(1)
    #else :
    df_question_3_p[(df_question_3_p.index.get_level_values('x_coord_bin')==x) & (df_question_3_p.index.get_level_values('y_coord_bin')==y)]=\
        df_question_3_p[(df_question_3_p.index.get_level_values('x_coord_bin')==x) & (df_question_3_p.index.get_level_values('y_coord_bin')==y)].apply(lambda x: x/alpha)
    #df_question_3[(df_question_3.index.get_level_values('x_coord')==x) & (df_question_3.index.get_level_values('y_coord')==y)][df_question_3.columns] - df_question_2.query(f'x_coord == {x} and y_coord == {y}')["mean"].values

In [16]:
df_question_3_p.tail(5)

name_team_that_shot      Anaheim Ducks  Arizona Coyotes  Boston Bruins  \
x_coord_bin y_coord_bin                                                  
100         10                     0.0              0.0            0.0   
            20                     NaN              NaN            NaN   
            30                     NaN              NaN            NaN   
            40                     NaN              NaN            NaN   
            50                     NaN              NaN            NaN   

name_team_that_shot      Buffalo Sabres  Calgary Flames  Carolina Hurricanes  \
x_coord_bin y_coord_bin                                                        
100         10                      0.0             0.0                  0.0   
            20                      NaN             NaN                  NaN   
            30                      NaN             NaN                  NaN   
            40                      NaN             NaN                  NaN   
            50                      NaN             NaN                  NaN   

name_team_that_shot      Chicago Blackhawks  Colorado Avalanche  \
x_coord_bin y_coord_bin                                           
100         10                          0.0                 0.0   
            20                          NaN                 NaN   
            30                          NaN                 NaN   
            40                          NaN                 NaN   
            50                          NaN                 NaN   

name_team_that_shot      Columbus Blue Jackets  Dallas Stars  \
x_coord_bin y_coord_bin                                        
100         10                             0.0           0.0   
            20                             NaN           NaN   
            30                             NaN           NaN   
            40                             NaN           NaN   
            50                             NaN           NaN   

name_team_that_shot      Detroit Red Wings  Edmonton Oilers  Florida Panthers  \
x_coord_bin y_coord_bin                                                         
100         10                         0.0              0.0               0.0   
            20                         NaN              NaN               NaN   
            30                         NaN              NaN               NaN   
            40                         NaN              NaN               NaN   
            50                         NaN              NaN               NaN   

name_team_that_shot      Los Angeles Kings  Minnesota Wild  \
x_coord_bin y_coord_bin                                      
100         10                         0.0             0.0   
            20                         NaN             NaN   
            30                         NaN             NaN   
            40                         NaN             NaN   
            50                         NaN             NaN   

name_team_that_shot      Montréal Canadiens  Nashville Predators  \
x_coord_bin y_coord_bin                                            
100         10                          0.0                  0.0   
            20                          NaN                  NaN   
            30                          NaN                  NaN   
            40                          NaN                  NaN   
            50                          NaN                  NaN   

name_team_that_shot      New Jersey Devils  New York Islanders  \
x_coord_bin y_coord_bin                                          
100         10                   16.060976                 0.0   
            20                         NaN                 NaN   
            30                         NaN                 NaN   
            40                         NaN                 NaN   
            50                         NaN                 NaN   

name_team_that_shot      New York Rangers  Ottawa Senators

In [17]:
df_question_3_p[df_question_3_p.isna()]=1

In [18]:
df_question_3_p

name_team_that_shot      Anaheim Ducks  Arizona Coyotes  Boston Bruins  \
x_coord_bin y_coord_bin                                                  
-100        -50               1.000000              1.0            1.0   
            -40               1.000000              1.0            1.0   
            -30               0.000000              0.0            0.0   
            -20               3.325758              0.0            0.0   
            -10               0.000000              0.0            0.0   
...                                ...              ...            ...   
100         10                0.000000              0.0            0.0   
            20                1.000000              1.0            1.0   
            30                1.000000              1.0            1.0   
            40                1.000000              1.0            1.0   
            50                1.000000              1.0            1.0   

name_team_that_shot      Buffalo Sabres  Calgary Flames  Carolina Hurricanes  \
x_coord_bin y_coord_bin                                                        
-100        -50                     1.0             1.0                  1.0   
            -40                     1.0             1.0                  1.0   
            -30                     0.0             0.0                  0.0   
            -20                     0.0             0.0                  0.0   
            -10                     0.0             0.0                  0.0   
...                                 ...             ...                  ...   
100         10                      0.0             0.0                  0.0   
            20                      1.0             1.0                  1.0   
            30                      1.0             1.0                  1.0   
            40                      1.0             1.0                  1.0   
            50                      1.0             1.0                  1.0   

name_team_that_shot      Chicago Blackhawks  Colorado Avalanche  \
x_coord_bin y_coord_bin                                           
-100        -50                         1.0                 1.0   
            -40                         1.0                 1.0   
            -30                         0.0                 0.0   
            -20                         0.0                 0.0   
            -10                         0.0                 0.0   
...                                     ...                 ...   
100         10                          0.0                 0.0   
            20                          1.0                 1.0   
            30                          1.0                 1.0   
            40                          1.0                 1.0   
            50                          1.0                 1.0   

name_team_that_shot      Columbus Blue Jackets  Dallas Stars  \
x_coord_bin y_coord_bin                                        
-100        -50                            1.0      1.000000   
            -40                            1.0      1.000000   
            -30                            0.0      0.000000   
            -20                            0.0      4.015244   
            -10                            0.0      0.000000   
...                                        ...           ...   
100         10                             0.0      0.000000   
            20                             1.0      1.000000   
            30                             1.0      1.000000   
            40                             1.0      1.000000   
            50                             1.0      1.000000   

name_team_that_shot      Detroit Red Wings  Edmonton Oilers  Florida Panthers  \
x_coord_bin y_coord_bin                                                         
-100        -50                        1.0         1.000000               1.0   
            -40                        1.0      

In [19]:
df_question_3_p.max().max()

16.0609756097561

In [20]:
df_question_3_p.to_csv(f"../datasets/data_to_plot_per_{season}.csv")
print(season)

20162017


In [90]:
def pivot(x):
    return -x

df = pd.read_csv('../datasets/tidy_data.csv')
df_season = df[df["season"]==20162017]

df_season.loc[(df_season['rinkSide_of_the_team_that_shot']=='') & (df_season['x_coord'] > 0),'rinkSide_of_the_team_that_shot'] = 'right' 

df_season.loc[df_season['rinkSide_of_the_team_that_shot']=='right','x_coord'] = df_season[df_season['rinkSide_of_the_team_that_shot']=='right']['x_coord'].apply(pivot)
df_season.loc[df_season['rinkSide_of_the_team_that_shot']=='right','y_coord'] = df_season[df_season['rinkSide_of_the_team_that_shot']=='right']['y_coord'].apply(pivot)

print(df_season[(df_season['x_coord'] > 0) &(df_season['rinkSide_of_the_team_that_shot']=='left') ]['rinkSide_of_the_team_that_shot'])


1        left
7        left
10       left
12       left
13       left
         ... 
80391    left
80392    left
80393    left
80397    left
80398    left
Name: rinkSide_of_the_team_that_shot, Length: 39413, dtype: object


In [43]:
def export_data_to_plot(season):
    
    def pivot(x):
        return -x
    
    df = pd.read_csv('../datasets/tidy_data.csv')
    df_season = df[df["season"]==season]
    
    side_to_rotate = 'left'
    if side_to_rotate == 'left':
        df_season.loc[(df_season['rinkSide_of_the_team_that_shot'].isna()) & (df_season['x_coord'] > 0),'rinkSide_of_the_team_that_shot'] = side_to_rotate 
    else:
        df_season.loc[(df_season['rinkSide_of_the_team_that_shot'].isna()) & (df_season['x_coord'] < 0),'rinkSide_of_the_team_that_shot'] = side_to_rotate 
        
    df_season.loc[df_season['rinkSide_of_the_team_that_shot']==side_to_rotate,'x_coord'] = df_season[df_season['rinkSide_of_the_team_that_shot']==side_to_rotate]['x_coord'].apply(pivot)
    df_season.loc[df_season['rinkSide_of_the_team_that_shot']==side_to_rotate,'y_coord'] = df_season[df_season['rinkSide_of_the_team_that_shot']==side_to_rotate]['y_coord'].apply(pivot)
    
    
    # average shot per hour per emplacement
    df_question_2 = df_season.groupby(['x_coord','y_coord'])['x_coord'].count().reset_index(name="count")
    df_question_2["mean_shots_per_hour_in_whole_league"] = df_question_2["count"]/df_season["id_game"].nunique() 

    # count per team per emplacement
    df_question_3 = df_season.groupby(['x_coord','y_coord','name_team_that_shot'])['name_team_that_shot'].count().reset_index(name="count_per_team")
    df_question_3_p = df_question_3.pivot(index=["x_coord","y_coord"], columns="name_team_that_shot", values="count_per_team")
    
    #df_question_3_p[df_question_3_p.isna()]=10e-14
    #print(df_question_3_p.head(5))


    # count game per team
    df_count_game_per_team = df_season.groupby(['name_team_that_shot'])["id_game"].nunique().reset_index(name="count_game_per_team")
    
    # compute probability of number of shots per emplacemnt per hour per team
    for team in df_question_3_p.columns :
        df_question_3_p[team] /= df_count_game_per_team[df_count_game_per_team["name_team_that_shot"]==team]["count_game_per_team"].values
    
    print("begin 2 minutes task")
    # percentage
    for x,y in  df_question_3_p.index:
        alpha = df_question_2.query(f'x_coord == {x} and y_coord == {y}')["mean_shots_per_hour_in_whole_league"].values
        df_question_3_p[(df_question_3_p.index.get_level_values('x_coord')==x) & (df_question_3_p.index.get_level_values('y_coord')==y)]=\
            df_question_3_p[(df_question_3_p.index.get_level_values('x_coord')==x) & (df_question_3_p.index.get_level_values('y_coord')==y)].apply(lambda x: x//alpha)
    
    df_question_3_p[df_question_3_p.isna()]=int(1)
    
    df_question_3_p.to_csv(f"../datasets/data_to_plot_per_{season}.csv")

    #return df_question_3_p
    

In [3]:
export_data_to_plot(20162017)

begin 2 minutes task


In [4]:
export_data_to_plot(20172018)

begin 2 minutes task


In [32]:
export_data_to_plot(20182019)

begin 2 minutes task


In [33]:
export_data_to_plot(20192020)

begin 2 minutes task


In [7]:
export_data_to_plot(20202021)

begin 2 minutes task
